In [1]:
suppressMessages(library(tidyverse))
suppressMessages(library(clusterProfiler))
suppressMessages(library("AnnotationDbi"))
suppressMessages(library("org.Hs.eg.db"))
library(msigdbr)

library(gage)
library(gageData)

data(kegg.sets.hs)
data(sigmet.idx.hs)

---
## Create signature table

In [2]:
sigs <- read.csv("./TB_signature_comparisons_all.csv") %>% mutate(value = 1)

sigs %>% 
    reshape2::dcast(formula = "List ~ Symbol", value.var = "value") %>% 
    t() %>% data.frame() %>% 
    write.table("./ALL_SIG_TABLE.csv", sep = ",", quote = F, row.names = T, col.names = F)

Aggregation function missing: defaulting to length



---
## Get KEGG, GO, and Hallmark pathways

In [7]:
### load deseq output
deseq_output = read.delim("./deseq_results (4).tsv") %>% mutate(subID = gsub("\\..*","",gene_id))
deseq_output %>% head()


### Add entrez ids
entrez = mapIds(org.Hs.eg.db,
                    keys= deseq_output$subID, #Column containing Ensembl gene ids
                    column="ENTREZID",
                    keytype="ENSEMBL",
                    multiVals="first")

entrez <- entrez %>% data.frame() 
colnames(entrez) <- "entrez"
deseq_output <- merge(deseq_output, entrez, by.x = "subID", by.y = 0,all.x=TRUE)

deseq_output %>% head()

,gene_id,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,gene_name,gene_type,subID
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>
1,ENSG00000223972.5,0.772045939,-0.6116703,0.3454366,-1.7707167,0.07660782,NA,DDX11L1,transcribed_unprocessed_pseudogene,ENSG00000223972
2,ENSG00000227232.5,0.524223968,-0.1007347,0.2737967,-0.3679178,0.71293456,NA,WASH7P,unprocessed_pseudogene,ENSG00000227232
3,ENSG00000278267.1,0.002551619,-0.5293662,2.9216896,-0.1811849,0.85622241,NA,MIR6859-1,miRNA,ENSG00000278267
4,ENSG00000243485.5,0.093088984,-0.3774343,1.0797869,-0.3495452,0.72668008,NA,MIR1302-2HG,lncRNA,ENSG00000243485
5,ENSG00000284332.1,0.000000000,NA,NA,NA,NA,NA,MIR1302-2,miRNA,ENSG00000284332
6,ENSG00000237613.2,0.062650816,-0.3624359,1.4960464,-0.2422624,0.80857681,NA,FAM138A,lncRNA,ENSG00000237613


'select()' returned 1:many mapping between keys and columns



,subID,gene_id,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,gene_name,gene_type,entrez
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>
1,ENSG00000000419,ENSG00000000419.14,17.538688,0.05683359,0.07020492,0.8095385,0.41820547,0.6762803,DPM1,protein_coding,8813
2,ENSG00000000457,ENSG00000000457.14,9.700100,-0.14844600,0.07844468,-1.8923655,0.05844230,0.2266374,SCYL3,protein_coding,57147
3,ENSG00000000460,ENSG00000000460.17,13.238671,-0.19583813,0.19131023,-1.0236678,0.30599219,0.5756441,C1orf112,protein_coding,55732
4,ENSG00000000938,ENSG00000000938.13,111.488393,0.13024416,0.07056582,1.8457118,0.06493407,0.2417467,FGR,protein_coding,2268
5,ENSG00000000971,ENSG00000000971.16,6.797092,-0.16201445,0.16668660,-0.9719704,0.33106527,0.5993624,CFH,protein_coding,3075
6,ENSG00000001036,ENSG00000001036.14,4.814409,0.14246335,0.13347059,1.0673763,0.28580194,0.5547198,FUCA2,protein_coding,2519


In [8]:
###-------------------------
### CREATE GENELIST
###-------------------------
## remove doubles
doubles <- deseq_output %>% group_by(entrez) %>% count() %>% filter(n>1) %>% pull(entrez)
deseq_output <- deseq_output %>% filter(!(entrez %in% doubles))

## create gene list
geneList <- as.numeric(deseq_output$log2FoldChange)
names(geneList) = as.character(deseq_output$entrez)
geneList = sort(geneList, decreasing = TRUE)


###-------------------------
### GO
###-------------------------
GO_res <- gseGO(geneList     = geneList,
              OrgDb        = org.Hs.eg.db,
              ont          = "BP",
              minGSSize    = 100,
              maxGSSize    = 500,
              pvalueCutoff = 0.05,
              verbose      = FALSE)

GO_res %>% head()


###-------------------------
### KEGG
###-------------------------
kegg.sets.hs = kegg.sets.hs[sigmet.idx.hs]

KEGG_res <- gage(geneList, gsets = kegg.sets.hs, same.dir=TRUE)
KEGG_res <- rbind(KEGG_res$greater %>% data.frame() %>% mutate(dir = "greater"),
                    KEGG_res$less %>% data.frame() %>% mutate(dir = "less")) %>% 
            data.frame()


###-------------------------
### HALLMARK
###-------------------------
H_t2g <- msigdbr(species = "Homo sapiens", category = "H") %>% 
  dplyr::select(gs_name, entrez_gene)
HM_res <- GSEA(geneList, TERM2GENE = H_t2g)


Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (1.45% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
Warning message in fgseaMultilevel(...):
“There were 444 pathways for which P-values were not calculated properly due to unbalanced (positive and negative) gene-level statistic values. For such pathways pval, padj, NES, log2err are set to NA. You can try to increase the value of the argument nPermSimple (for example set it nPermSimple = 10000)”
Warning message in fgseaMultilevel(...):
“For some of the pathways the P-values were likely overestimated. For such pathways log2err is set to NA.”
Warning message in fgseaMultilevel(...):
“For some pathways, in reality P-values are less than 1e-10. You can set the `eps` argument to zero for better estimation.”


,ID,Description,setSize,enrichmentScore,NES,pvalue,p.adjust,qvalues,rank,leading_edge,core_enrichment
,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
GO:0000070,GO:0000070,mitotic sister chromatid segregation,162,0.4008520,2.251412,1e-10,1.965703e-09,5.802814e-10,5915,"tags=35%, list=19%, signal=28%",701/285643/9232/11004/81620/9700/83540/113130/5347/4751/9055/79075/990/9212/11065/699/11130/10615/7272/29127/51529/64151/55143/27127/9787/83903/3833/642636/9493/151648/3834/146909/10540/54892/9555/991/4085/9735/10300/10459/10403/26271/151246/7283/1843/79998/1062/5119/84722/90417/10460/128866/3837/9587/23636/56984
GO:0000819,GO:0000819,sister chromatid segregation,189,0.3825177,2.201965,1e-10,1.965703e-09,5.802814e-10,5915,"tags=33%, list=19%, signal=27%",701/285643/9232/11004/81620/9700/83540/113130/5347/4751/9055/79075/990/9212/11065/7153/699/11130/10615/7272/29127/2237/51529/116028/150280/64151/55143/27127/9787/83903/3833/642636/9493/151648/3834/146909/10734/157570/10540/79892/54892/9555/991/4085/9735/10300/10459/10403/26271/151246/7283/1843/79998/1062/5119/84722/90417/10460/128866/3837/9587/23636/56984
GO:0001505,GO:0001505,regulation of neurotransmitter levels,200,0.3851766,2.202084,1e-10,1.965703e-09,5.802814e-10,5681,"tags=42%, list=18%, signal=35%",3766/116443/6539/2572/9152/440279/143425/2914/590/8825/8499/145270/6861/6507/25953/5582/6512/2901/8618/6582/9699/120892/815/339302/5865/3176/6511/27445/26002/148/5413/8447/8775/1312/6529/148281/23621/808/1812/11315/2571/8676/150/783/80736/43/9378/41/10211/90019/23396/3690/2555/8541/3688/8773/222962/6506/10497/9342/6854/1103/79772/5538/6530/477/55784/5023/6809/9379/5071/23413/63908/2030/112755/84062/8224/3356/2670/23557/6804/100289017/23208/6813
GO:0001906,GO:0001906,cell killing,175,0.4222374,2.375054,1e-10,1.965703e-09,5.802814e-10,4532,"tags=33%, list=15%, signal=28%",1669/1668/1991/566/1511/8993/820/4057/383/81494/735/3606/6283/4069/634/3105/1113/3151/7305/3805/57823/149233/3952/3106/3135/3916/10383/8807/2597/4589/8970/3133/59067/3078/6688/80329/64581/3140/140823/409/909/85236/9437/146850/3684/4615/10859/3902/201294/3594/5027/4843/3592/11035/3965/3273/57817/5196
GO:0002221,GO:0002221,pattern recognition receptor signaling pathway,158,0.4293770,2.392785,1e-10,1.965703e-09,5.802814e-10,6408,"tags=44%, list=21%, signal=35%",4057/57402/26253/9971/2867/3659/2672/3304/3430/7301/2219/92610/10062/2099/23643/4814/55765/80216/57590/64127/285852/26007/6565/1520/57864/11314/3303/1535/7867/409/9111/10211/8638/929/9261/55366/4615/4940/353376/8767/11213/3665/56927/118788/79671/81570/64135/7353/388325/3929/7096/114609/338382/11027/7100/4067/146722/84282/7099/25921/6397/7097/3146/81035/10333/4938/130120/23586/148022
GO:0002833,GO:0002833,positive regulation of response to biotic stimulus,155,0.4426869,2.480737,1e-10,1.965703e-09,5.802814e-10,5503,"tags=40%, list=18%, signal=33%",115362/29126/383/9447/5225/338322/2867/58484/117854/3430/2219/7305/3805/2358/400709/4332/3135/3916/83666/5359/8807/23643/5322/3133/6714/59067/29108/6688/55122/64127/64581/3140/1535/5058/9111/8638/3148/3902/2896/3665/3592/6352/3273/388325/3929/3263/3134/3055/5819/114609/354/3593/11027/3428/4067/115004/57506/84166/7099/3320/5578/81030


preparing geneSet collections...

GSEA analysis...

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (1.45% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”
Warning message in fgseaMultilevel(...):
“There were 10 pathways for which P-values were not calculated properly due to unbalanced (positive and negative) gene-level statistic values. For such pathways pval, padj, NES, log2err are set to NA. You can try to increase the value of the argument nPermSimple (for example set it nPermSimple = 10000)”
Warning message in fgseaMultilevel(...):
“For some pathways, in reality P-values are less than 1e-10. You can set the `eps` argument to zero for better estimation.”
leading edge analysis...

done...



In [15]:
GO_res %>% write.csv("./2024.01.04_GO-RES.csv",quote=FALSE, row.names=TRUE)
KEGG_res %>% write.csv("./2024.01.04_KEGG-RES.csv",quote=FALSE, row.names=TRUE)
HM_res %>% write.csv("./2024.01.04_HM-RES.csv",quote=FALSE, row.names=TRUE)